In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xgboost as xgb
import numpy as np
import pandas as pd
import tensorflow as tf
"""
'inc_ability': 考虑到您的能力和工作状况，您目前的收入是否合理
'view': 根据您的一般印象您对一些重要事情所持的观点和看法与社会大众一致的时候有多少 
'status_3_before': 与三年前相比，您的社会经济地位发生了什么变化
'status_peer': 与同龄人相比，您本人的社会经济地位怎样
"""

"\n'inc_ability': 考虑到您的能力和工作状况，您目前的收入是否合理\n'view': 根据您的一般印象您对一些重要事情所持的观点和看法与社会大众一致的时候有多少 \n'status_3_before': 与三年前相比，您的社会经济地位发生了什么变化\n'status_peer': 与同龄人相比，您本人的社会经济地位怎样\n"

In [2]:
data_dir = '/Users/changqingli/pycharm_projects/algorithms/weekend_content/'
train_data_file = 'data/happiness_train_abbr.csv'
test_data_dile = 'data/happiness_test_abbr.csv'

In [3]:
df_train = pd.read_csv(data_dir+train_data_file)
df_test = pd.read_csv(data_dir+test_data_dile)
df_train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,...,family_income,family_m,family_status,house,car,marital,status_peer,status_3_before,view,inc_ability
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,...,60000.0,2,2,1,2,3,3,2,4,3
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,...,40000.0,3,4,1,2,1,1,1,4,2
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,...,8000.0,3,3,1,2,3,2,1,4,2
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,...,12000.0,3,3,1,1,7,2,1,3,2
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,...,-2.0,4,3,1,1,1,3,2,3,-8


In [4]:
df_train = df_train[['happiness', 'inc_ability', 'view', 'status_3_before', 'status_peer']]
df_test = df_test[['inc_ability', 'view', 'status_3_before', 'status_peer']]

In [6]:


# df_train = df_train.drop('survey_time', 1).drop('id', 1)
df_train = df_train[df_train['happiness'] != -8]
df_train = df_train.fillna(0)

# df_test = df_test.drop('survey_time', 1)
df_test = df_test.fillna(0)



In [7]:
df_train_train = df_train.sample(frac=0.8, random_state=0, axis=0)
df_train_validation = df_train[~df_train.index.isin(df_train_train.index)]



In [8]:
tr_data = df_train_train.as_matrix()
va_data = df_train_validation.as_matrix()
te_data = df_test.as_matrix()
tr_data


/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/changqingli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[ 3, -8,  3,  2,  3],
       [ 5,  2,  4,  1,  3],
       [ 5,  2,  4,  1,  2],
       ...,
       [ 4,  2,  3,  2,  2],
       [ 3,  3,  4,  2,  3],
       [ 5,  2,  4,  2,  3]])

In [9]:
tr_x_data = tr_data[:, 1:]
tr_y_data = tr_data[:, 0]

va_x_data = va_data[:, 1:]
va_y_data = va_data[:, 0]

In [10]:
len(va_x_data)

1598

In [11]:
from sklearn import linear_model
mod = linear_model.LinearRegression()

In [12]:
mod.fit(tr_x_data, tr_y_data)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
va_y_pred = np.round(mod.predict(va_x_data))

In [14]:
print(va_y_data)
print(va_y_pred)
print(va_y_pred - va_y_data)
print(np.mean((va_y_pred - va_y_data)*(va_y_pred - va_y_data)))

[4 4 5 ... 3 5 4]
[4. 4. 4. ... 3. 4. 4.]
[ 0.  0. -1. ...  0. -1.  0.]
0.6958698372966208


In [17]:
pred_data = te_data
pred_data

array([[ 2,  4,  1,  3],
       [ 2,  4,  3,  3],
       [ 4,  3,  1,  3],
       ...,
       [ 2,  4,  1,  2],
       [-8,  4,  1,  2],
       [ 2,  4,  2,  2]])

In [18]:
pred_res = np.round(mod.predict(te_data))

In [19]:
pred_res

array([4., 4., 4., ..., 4., 4., 4.])

In [20]:
import csv
result = 'submission_908_less_feature.csv'
with open(data_dir+result, 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['id', 'happiness'])
    for i in range(len(te_data)):
        writer.writerow([int(te_data[i][0]), int(pred_res[i])])

In [21]:
len(mod.coef_)

4

In [22]:
col_name = list(df_train_train.columns)[1:]
m = {}
for i in range(len(col_name)):
    m.setdefault(col_name[i], 0)
    m[col_name[i]] = mod.coef_[i]
    
sorted(m.items(), key=lambda x: x[1], reverse=True)
    

[('view', 0.03608262448051032),
 ('inc_ability', -0.006116068180416026),
 ('status_3_before', -0.01758004832476882),
 ('status_peer', -0.11678145677539911)]